In [35]:
import re
import nbformat as nbf

def rmd_to_notebook(rmd_file, notebook_file):
    # Read the R Markdown file
    with open(rmd_file, 'r') as file:
        rmd_lines = file.readlines()
    
    # Preview the first 100 lines of the R Markdown file
    print("Preview of the first 100 lines of the R Markdown file:")
    for i, line in enumerate(rmd_lines[:100]):
        print(f"{i+1}: {line}", end='')
    print("\n")
    
    rmd_content = ''.join(rmd_lines)
    
    # Initialize a new notebook
    nb = nbf.v4.new_notebook()
    cells = []

    # Extract the title from the YAML front matter
    title_match = re.search(r'^---\s*title:\s*"(.*?)".*?---', rmd_content, flags=re.DOTALL | re.MULTILINE)
    if title_match:
        title = title_match.group(1)
        cells.append(nbf.v4.new_markdown_cell(f"# {title}"))
    
    # Remove the YAML front matter
    rmd_content = re.sub(r'^---.*?---', '', rmd_content, flags=re.DOTALL | re.MULTILINE)
    
    # Remove instructor notes and solutions
    rmd_content = re.sub(r':::\s*(instructor|solution)\s*.*?:::', '', rmd_content, flags=re.DOTALL)

    # Define regex patterns
    header_pattern = re.compile(r'^(#+) (.*)', re.MULTILINE)
    block_start_pattern = re.compile(r'^::+(\s*\w+)?')
    block_end_pattern = re.compile(r'^::+')
    
    # Split the content into lines
    lines = rmd_content.split('\n')
    
    code_buffer = []
    is_in_code_block = False
    text_buffer = []
    is_in_special_block = False
    special_block_type = ""

    def process_buffer(buffer):
        if buffer:
            cells.append(nbf.v4.new_markdown_cell('\n'.join(buffer)))
            buffer.clear()

    for line in lines:
        if line.startswith('```'):
            is_in_code_block = not is_in_code_block
            if is_in_code_block:
                process_buffer(text_buffer)
            else:
                if code_buffer:
                    cells.append(nbf.v4.new_code_cell('\n'.join(code_buffer).strip()))
                    code_buffer = []
        elif is_in_code_block:
            code_buffer.append(line)
        else:
            header_match = header_pattern.match(line)
            block_start_match = block_start_pattern.match(line)
            block_end_match = block_end_pattern.match(line)

            if header_match:
                process_buffer(text_buffer)
                header_level = len(header_match.group(1))
                header_text = header_match.group(2)
                cells.append(nbf.v4.new_markdown_cell(f"{'#' * header_level} {header_text}"))
            elif block_start_match:
                process_buffer(text_buffer)
                if block_start_match.group(1):
                    is_in_special_block = True
                    special_block_type = block_start_match.group(1).strip().capitalize()
                    text_buffer.append(f"### {special_block_type}")
            elif block_end_match and is_in_special_block:
                process_buffer(text_buffer)
                is_in_special_block = False
            elif not block_end_match:  # Ignore lines with only colons
                text_buffer.append(line)
    
    process_buffer(text_buffer)

    # Print the cells
    print("Cells in the notebook:")
    for cell_ct, cell in enumerate(cells[:40]):
        print(f"{cell_ct}: {cell}")
        
    print("\n")

    # Add cells to the notebook
    nb['cells'] = cells
    
    # Write the notebook to a file
    with open(notebook_file, 'w') as file:
        nbf.write(nb, file)

# Use the function to convert an R Markdown file to a Jupyter notebook
rmd_to_notebook('test-markdown-to-jupyter.Rmd', 'test-markdown-to-jupyter.ipynb')


Preview of the first 100 lines of the R Markdown file:
1: ---
2: title: "Monitor the training process"
3: teaching: 120
4: exercises: 80
5: ---
6: 
7: ::: questions
8: - How do I create a neural network for a regression task?
9: - How does optimization work?
10: - How do I monitor the training process?
11: - How do I detect (and avoid) overfitting?
12: - What are common options to improve the model performance?
13: :::
14: 
15: ::: objectives
16: - Explain the importance of keeping your test set clean, by validating on the validation set instead of the test set
17: - Use the data splits to plot the training process
18: - Explain how optimization works
19: - Design a neural network for a regression task
20: - Measure the performance of your deep neural network
21: - Interpret the training plots to recognize overfitting
22: - Use normalization as preparation step for Deep Learning
23: - Implement basic strategies to prevent overfitting
24: :::
25: 
26: ::: instructor
27: ## Copy-pasting 